In [ ]:
def egcd_pade_canonico(q, pol_f, pol_g, k):
    """
    EEA 'canónico' para Padé sobre R = Zmod(q)[x].

    ENTRADA:
      q      : entero (típicamente primo). Trabajamos en Z/qZ.
      pol_f  : coeficientes de f (por ejemplo [0,0,0,0,1] para x^4).
      pol_g  : coeficientes de g (por ejemplo [1,2,3,4] para 1+2x+3x^2+4x^3).
      k      : umbral para detenerse en la primera fila j con deg(r_j) < k.

    SALIDA:
      ("ok", r_can, t_can, j)    si existe el Padé (k, n-k), con t_can mónico,
      ("unsat", rj, tj, j)       si el sistema NO es soluble (certificado).

    Notas:
      - Se preservan las invariantes r_i = s_i f + t_i g.
      - Cada fila se normaliza para que lc(r_i) = 1 y se propaga a s_i, t_i.
      - Requiere que lc(f), lc(g) sean invertibles en Z/qZ.
    """
    R = PolynomialRing(Zmod(q), 'x'); x = R.gen()
    f = R(pol_f)
    g = R(pol_g)

    if f.is_zero() or g.is_zero():
        raise ValueError("f y g deben ser no nulos.")

    # Normalización inicial y (s0,t0), (s1,t1) correctos
    inv_lc_f = 1 / f.leading_coefficient()
    inv_lc_g = 1 / g.leading_coefficient()

    r = [f * inv_lc_f, g * inv_lc_g]
    s = [R(inv_lc_f), R(0)]
    t = [R(0),         R(inv_lc_g)]

    i = 1
    while True:
        # Criterio de parada: primera fila con deg(r_i) < k
        if r[i].is_zero():
            # nos quedamos con la última fila válida i-1
            j = i - 1
            rj, tj = r[j], t[j]
            if rj.degree() < k and rj.gcd(tj) == 1:
                tau = tj.leading_coefficient()
                return ("ok", rj / tau, tj / tau, j)
            return ("unsat", rj, tj, j)

        if r[i].degree() < k:
            j = i
            rj, tj = r[j], t[j]
            if rj.gcd(tj) == 1:
                tau = tj.leading_coefficient()
                return ("ok", rj / tau, tj / tau, j)
            return ("unsat", rj, tj, j)

        # Paso EEA
        qi, ri1 = r[i-1].quo_rem(r[i])
        si1 = s[i-1] - qi * s[i]
        ti1 = t[i-1] - qi * t[i]

        # Normalización de la nueva fila: lc(r_{i+1}) = 1 y propagar a s,t
        if not ri1.is_zero():
            inv_lc = 1 / ri1.leading_coefficient()
            ri1 *= inv_lc
            si1 *= inv_lc
            ti1 *= inv_lc

        r.append(ri1)
        s.append(si1)
        t.append(ti1)
        i += 1


# ===== Ejemplo de uso =====
# Padé (k=2, n-k=2) sobre F_5 con f = x^4 y g = 1+2x+3x^2+4x^3
res = egcd_pade_canonico(5, [0,0,0,0,1], [1,2,3,4], k=2)
print(res)
# Esperado: ("ok", r_can, t_can, j) con t_can mónico, y r_can/t_can el Padé (2,2).


In [ ]:
('ok', 1, x^2 + 3*x + 1, 3)

**estado:** ok

\( r_{\text{can}} = 1 \)

\( t_{\text{can}} = x^{2} + 3x + 1 \in \mathbb{F}_5[x] \)

\( j = 3 \)

Es exactamente el Padé \((k, n-k) = (2, 2)\) esperado.



**Chequeo rápido (en \(\mathbb{F}_5\), módulo \(x^4\)):**

\( t \cdot g = (x^2 + 3x + 1)(1 + 2x + 3x^2 + 4x^3) \equiv 1 \pmod{x^4} \),

porque al expandir:
- término constante: \(1\);
- coeficientes de \(x, x^2, x^3\) suman \(0 \bmod 5\);
- términos \(\ge x^4\) se anulan \(\bmod x^4\).

Así \( r \equiv t\,g \; (\bmod x^4) \) con \(\deg r = 0 < k = 2\) y \(\deg t = 2 \le n-k = 2\).
Además \(\gcd(r,t) = \gcd(1,t) = 1\), y el denominador ya quedó mónico, así que estás en forma canónica.

**Interpretación:** el aproximante de Padé es
\[
\frac{r}{t} = \frac{1}{x^2 + 3x + 1} \quad \text{en } \mathbb{F}_5(x).
\]
